# Aula 06 - Dados nD - netCDF

Nesta aula trabalhamos com dados 3 ou 4 D em formato netCDF. 
Utilizamos o pacote https://docs.xarray.dev/en/stable/ e  trabalharemos com os dados da plataforma
https://disc.gsfc.nasa.gov/

## Baixando os dados do GESDISC
- Crie um login para você
- Digite uma palavra-chave que descreva o dado que desejar
- Baixe o dado para um determinado período
- https://disc.gsfc.nasa.gov/datasets/MERRA2_CNN_HAQAST_PM25_1/summary?keywords=air%20quality

